## Moat Brand Reporting Refresh

In [1]:
from rtf_utils.moat_utils import MoatTile
from essence.analytics.platform import securedcredentials as secure_creds
from rtf_utils.gcp_utils import BigQuery,CloudStorage
import warnings
from rtf_utils.bq_schema import moat_schema_dict

warnings.filterwarnings("ignore", "Your application has authenticated using end user credentials")
import json

In [2]:
service_account_email='131786951123-compute@developer.gserviceaccount.com' ## gcp analytics service account
PROJECT_ID='essence-analytics-dwh'

credentialsFromVault=secure_creds.getCredentialsFromEssenceVault(service_account_email)
print("Creds Loaded")
moat_token = secure_creds.getDataFromEssenceVault('Moat_Token_Google')
print("Token Loaded")

Creds Loaded
Token Loaded


In [3]:
start_date = '20190701'
end_date = '20191001'

In [1]:
with open('dags/brand_report_task_queue.json') as json_file:
    config_json = json.load(json_file)
    campaigns = config_json['campaigns']

NameError: name 'json' is not defined

In [4]:
bonito = [(13120,{'level1':10154328017481183},['date','level1','level4']),
          (13386,{'level1':7020493427},['date','level1','level4'])]

In [5]:
def get_moat_data(tile_id,start_date, end_date,level_filters=None,dimensions=None,**kwargs):
    tile = MoatTile(tile_id,level_filters,dimensions)
    local_filename = tile.get_data(start_date,end_date,moat_token)
    return local_filename

def upload_local_file(local_filename,dest_bucket,folder=None):
    gcs = CloudStorage(credentialsFromVault)
    if folder:
        blob_name = folder + "/" + local_filename
    else:
        blob_name = local_filename
    blob = gcs.upload_blob(dest_bucket, blob_name,local_filename,mode='filename')
    return blob

# Prototype WorkFlow

In [22]:
tile_id, level_filter, dimensions = (13120,{'level1':10154328017481183},['date','level1','level4'])

In [23]:
filename = get_moat_data(tile_id,'20190801','20190901',level_filter,dimensions)

Data Available, Storing


In [ ]:
filename

In [9]:
b = upload_local_file(filename,"rtf_staging",folder="brand_reporting")

In [6]:
import os
os.environ['GOOGLE_CLOUD_PROJECT'] = 'essence-analytics-dwh'
for x in bonito:
    tile_id, level_filter, dimensions = x
    print(tile_id, level_filter, dimensions)
    filename = get_moat_data(tile_id,start_date,end_date,level_filter,dimensions)
    print("Stored at {}".format(filename))    
    file_uri = upload_local_file(filename,"rtf_staging",folder="brand_reporting")
    
    file_uri = "gs://rtf_staging/" + file_uri.name
    
    print("Load Schema")
    #schema_dict = moat_schema_dict[tile_id]
    
    dest_table = "{}_{}".format("NewMoat",tile_id)
    
    print("Try BQ Upload")
        
    bq = BigQuery(credentialsFromVault)
    resp = bq.load_from_gcs("rtf_brand_reporting",
                     file_uri,
                    dest_table,
                    #schema=schema_dict['schema'],
                    extension='json')
    
    
    
    

13120 {'level1': 10154328017481183} ['date', 'level1', 'level4']
Data Available, Storing
Stored at Moat_13120_10154328017481183.json
Load Schema
Try BQ Upload
13386 {'level1': 7020493427} ['date', 'level1', 'level4']
Data Available, Storing
Stored at Moat_13386_7020493427.json
Load Schema
Try BQ Upload


In [3]:
MoatTile

rtf_utils.moat_utils.MoatTile

In [13]:
import requests
auth_header = 'Bearer {}'.format(moat_token)

fields = ["impressions_analyzed",
                    "loads_unfiltered",
                    "susp_human",
                    "human_and_viewable",
                    ] 

query = {
                'metrics': ','.join(['level1']),
                'start': start_date,
                'end': end_date,
                'brandId':13120, ## this is the tile ID 
                } 


resp = requests.get('https://api.moat.com/1/stats.json',
                        params=query,
                        headers={'Authorization': auth_header,
                                    'User-agent': 'Essence Global 1.0'}
                       )
r = resp.json()

In [14]:
r

{'data_available': True,
 'query': {'end': '20191001', 'metrics': 'level1', 'start': '20190918'},
 'results': {'details': [{'level1_id': '1376497225955242',
    'level1_label': 'Google | NA | USD | GSA'},
   {'level1_id': '334868853997818',
    'level1_label': 'Google | EMEA | GBP | Multi App'},
   {'level1_id': '10154962304896183',
    'level1_label': 'Google | EMEA | GBP | Home | GB'},
   {'level1_id': '10155942789716183',
    'level1_label': 'Google | EMEA | USD | Chromebook'},
   {'level1_id': '10154328017481183',
    'level1_label': 'Google | NA | USD | Pixel'},
   {'level1_id': '599549490505228',
    'level1_label': 'Google | EMEA | USD | Google Ads'},
   {'level1_id': '645954942417705',
    'level1_label': 'Google | EMEA | USD | YouTube Music | Brand'},
   {'level1_id': '10155755517181183',
    'level1_label': 'Google | EMEA | USD | Google My Business'},
   {'level1_id': '10154379737626183',
    'level1_label': 'Google | EMEA | GBP | Pixel | UK'},
   {'level1_id': '1015559114856